In [70]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib import_ipynb

In [71]:
from fastai.tabular.all import *
from pathlib import Path
import import_ipynb
import ipynb
import stockFetcher

In [72]:
testFolder = Path.cwd().parent / 'TestData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

In [73]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
countryGuesser.pkl
stockScreenerV1.5.pkl
animalClassifier.pkl
stockScreenerV1.4.pkl
stockScreenerV1.6.pkl
leagueChampionClassifier.pkl
stockScreenerV1.7.pkl
politicianClassifier.pkl
stockScreenerV1.3.pkl
stockScreenerV1.2.pkl
stockScreenerV1.0.pkl
stockScreenerV1.1.pkl
test.pkl
stockScreenerV1.9.pkl
stockScreenerV1.8.pkl
stockScreenerV1.10.pkl


In [78]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[1]
bestModel.head()

Model Name    stockScreenerV1.5
Timestamp      2025-01-19 10:53
MAE                       0.285
RMSE                      0.362
R2                       -0.996
Name: 5, dtype: object

In [75]:
modelName = f"{bestModel['Model Name']}.pkl" # Change this if you want to try other models
learn = load_learner(modelFolder / modelName)

In [76]:
predictionTarget = 'Best' # 'Best' for the best performing tickers 

if predictionTarget == 'Best':
    adr_df = pd.read_csv(testFolder / 'adrTickers.csv')
    symbols = adr_df['Ticker'].tolist()
    test_df = [stockFetcher.getTickerData(symbol) for symbol in symbols]
    test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame

else:
    test_df = stockFetcher.getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
    test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

$TOT: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for TOT: single positional indexer is out-of-bounds


$ABB: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for ABB: single positional indexer is out-of-bounds


$LFC: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


...

Error fetching data for CN: single positional indexer is out-of-bounds


$MTL: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


Error fetching data for MTL: single positional indexer is out-of-bounds


/var/folders/ch/yfh31z5s3cq4cw_md59n4qw40000gn/T/ipykernel_81972/406433226.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' 

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,83.199997,85.589996,83.029999,85.120003,21089700,0.0,0.0,-0.045255,2.023515e+11,-17.456805,Internet Retail
1,48.700001,49.840000,48.599998,49.340000,4331600,0.0,0.0,NaN,4.487946e+11,NaN,Internet Content & Information
2,215.169998,215.679993,211.059998,211.500000,21559000,0.0,0.0,-0.256812,1.096955e+12,-3.076181,Semiconductors
3,60.939999,61.790001,60.680000,61.099998,4696900,0.0,0.0,9.970887,7.655341e+10,0.079231,Other Industrial Metals & Mining
4,4.890000,4.930000,4.870000,4.890000,1891200,0.0,0.0,5.237325,7.407176e+10,0.150840,Banks - Diversified


In [77]:
if predictionTarget == 'Best':
    prediction = learn.get_preds(dl=dl)
    sorted_predictions = sorted(zip(symbols, prediction[0]), key=lambda x: x[1], reverse=True)
    print(f"Prediction for best performing tickers:")
    for symbol, pred in sorted_predictions:
        print(f"{symbol}: {pred[0].item() * 100:.2f}%")
else:
    prediction = learn.get_preds(dl=dl)
    print(f"Prediction for {predictionTarget}:")
    print(f"{prediction[0][0][0].item() * 100:.2f}%")

Prediction for best performing tickers:
TEVA: 83500.68%
BABA: 812.53%
PHG: 589.91%
CS: 254.85%
CNHI: 90.07%
BHP: 74.88%
PKX: 73.22%
HMC: 63.86%
PTR: 63.76%
RELX: 63.74%
CHU: 58.59%
MT: 54.31%
GSK: 54.24%
CX: 53.54%
NTES: 53.32%
LFC: 51.36%
DEO: 47.04%
ENB: 44.43%
FANUY: 35.62%
KEP: 29.82%
LGGNY: 29.52%
UL: 28.68%
SAN: 28.10%
RACE: 27.80%
INFY: 26.34%
AMX: 20.19%
BIDU: 17.96%
BNS: 17.51%
RY: 17.35%
TM: 13.29%
ZTO: 13.26%
ERIC: 12.31%
ITUB: 10.81%
LVMUY: 9.72%
SNY: 8.08%
VALE: 7.71%
ASML: 6.91%
NVS: 4.94%
VOD: 4.69%
CM: 3.92%
VWAGY: 3.04%
ABB: 2.88%
BBVA: 2.88%
STM: 2.36%
BASFY: 1.76%
SLB: 1.39%
ENIC: -0.58%
NTTYY: -0.97%
AZN: -1.55%
SAP: -3.10%
MFG: -3.10%
HSBC: -3.24%
CEO: -4.44%
BP: -6.26%
SONY: -6.57%
RDS-B: -6.77%
JD: -7.00%
SIEGY: -9.21%
RDS-A: -9.49%
TCEHY: -11.56%
ORAN: -11.89%
RIO: -12.08%
IBN: -13.58%
TSM: -13.97%
SMFG: -14.03%
CLPBY: -16.74%
TOT: -16.87%
VALE: -19.16%
PDD: -20.65%
TD: -21.52%
NSANY: -26.68%
ABBV: -27.84%
